In [15]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import Ridge

In [2]:
train = pd.read_csv('salary-train.csv')
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [3]:
test = pd.read_csv('salary-test-mini.csv')
test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [6]:
train.FullDescription = train.FullDescription.replace('[^a-zA-Z0-9]', ' ', regex=True).str.lower()
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,NaN,20355


In [7]:
vectorizer = TfidfVectorizer(min_df=5)
tf_idf = vectorizer.fit_transform(train.FullDescription)

In [8]:
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


In [9]:
one_hot_vectorizer = DictVectorizer()
one_hot = one_hot_vectorizer.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [13]:
x = hstack([tf_idf, one_hot])
y = train['SalaryNormalized']

In [16]:
clf = Ridge(alpha=1, random_state=241)
clf.fit(x, y)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=241, solver='auto', tol=0.001)

In [18]:
tf_idf_test = vectorizer.transform(test.FullDescription)
one_hot_test = one_hot_vectorizer.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
x_test = hstack([tf_idf_test, one_hot_test])

res = clf.predict(x_test)
print(res)

[56555.61500155 37188.32442618]
